In [1]:
import numpy as np

For the given function, the Hessian matrix is \
\
$ \begin{bmatrix}  800 & 0 \\ 0 & 0.048x_2^2 \end{bmatrix} $

In [2]:
def evalh(x): 
  assert type(x) is np.ndarray 
  assert len(x) == 2
  return np.array([[800,0] , [0,0.048*x[1]**2]])

In [3]:
def find_condition_number(A):
  assert type(A) is np.ndarray
  assert A.shape[0] == A.shape[1]
  a,b = np.linalg.eig(A)
  return max(a)/min(a)

In [4]:
def evalf(x):  
  assert type(x) is np.ndarray and len(x) == 2 
  return 400*x[0]**2 + 0.004*x[1]**4

In [5]:
def evalg(x):  
  assert type(x) is np.ndarray and len(x) == 2
  return np.array([800*x[0],0.016*x[1]**3])

In [6]:
def compute_D_k(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  hsn_mtrx = evalh(x)
  return np.array([[1/hsn_mtrx[0][0], 0], [0, 1/hsn_mtrx[1][1]]])

In [7]:
def compute_D_k_1(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  if np.linalg.det(evalh(x)) == 0:
    raise ValueError('Determinant is 0 ')
  return np.linalg.inv(evalh(x))  

In [8]:
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  
  #Complete the code 
  alpha = alpha_start
  p_k = -gradf
  #implement the backtracking line search
  while evalf(x + alpha*p_k) > (evalf(x) + gamma*alpha*(np.dot(gradf.transpose(),p_k))):
    alpha = rho* alpha 
  return alpha
  

In [9]:
def compute_steplength_backtracking_scaled_direction(x, gradf, direction, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(direction) is np.ndarray and len(direction) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  alpha = alpha_start
  while evalf(x + alpha*direction) > ( evalf(x) +  (gamma*alpha*np.dot(gradf,direction)) ):
    alpha = rho*alpha
  return alpha

In [10]:
BACKTRACKING_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH_SCALING = 2
BACKTRACKING_LINE_SEARCH_SCALED = 3
CONSTANT_STEPLENGTH = 4

In [11]:
def find_minimizer_gd(start_x, tol, line_search_type,*args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  
  x = start_x
  g_x = evalg(x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    #print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)
  condition_number=[]
  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    condition_number.append(find_condition_number(evalh(x)))
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == CONSTANT_STEPLENGTH:
      step_length = 1.0

    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x ,evalf(x),k,condition_number


In [12]:
#complete the code for gradient descent with scaling to find the minimizer

def find_minimizer_gdscaling(start_x, tol, line_search_type,*args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  x = start_x
  g_x = evalg(x)
  d_k = compute_D_k(x)
  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH_SCALING):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    #print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    direction = -np.matmul(d_k,g_x)
    if line_search_type == BACKTRACKING_LINE_SEARCH_SCALING:
      step_length = compute_steplength_backtracking_scaled_direction(x,g_x,direction,alpha_start,rho,gamma)
    elif line_search_type == CONSTANT_STEPLENGTH:
      step_length = 1.0

    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.matmul(np.multiply(step_length,g_x),d_k)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x ,evalf(x),k


In [13]:
import scipy
from scipy.linalg import sqrtm

In [14]:
def find_minimizer_gdscaling_non_diagonal(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  x = start_x
  g_x = evalg(x)

  condition_number_scaling = []
  

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH_SCALED):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
  k = 0
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    d_k = compute_D_k_1(x)

    if line_search_type == BACKTRACKING_LINE_SEARCH_SCALED:
      
      direction = np.matmul(-d_k,g_x)
      
      p = scipy.linalg.sqrtm(d_k)
      hessn = np.matmul(np.matmul(p,evalh(x)),p)
      condition_number_scaling.append(find_condition_number(hessn))
      step_length = compute_steplength_backtracking_scaled_direction(x, g_x,direction, alpha_start, rho, gamma) #call the new function you wrote to compute the steplength 
    elif line_search_type == CONSTANT_STEPLENGTH:
      step_length = 1.0
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here  
    x = np.subtract(x, np.multiply(step_length,np.matmul(d_k, g_x))) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point
    

  return x ,evalf(x),k,condition_number_scaling

$ \huge{3.}$

In [15]:
tol = 1e-6
alpha = 1.0
rho = 0.5
gamma = 0.5
start = np.array([2.0,2.0])

In [16]:
x,f_x,no_of_iterations,condition_no = find_minimizer_gdscaling_non_diagonal(start,tol,CONSTANT_STEPLENGTH,alpha,rho,gamma)

In [17]:
print("Newtons method with scaling and constant step length")
print("Minimizer : ",x)
print("Function values : ",f_x)
print("Number of iteration : ",no_of_iterations)

Newtons method with scaling and constant step length
Minimizer :  [0.         0.03468306]
Function values :  5.788014517642641e-09
Number of iteration :  10


In [18]:
x,f_x,no_of_iterations,condition_no = find_minimizer_gdscaling_non_diagonal(start,tol,BACKTRACKING_LINE_SEARCH_SCALED,alpha,rho,gamma)


In [19]:
print("Newtons method with scaling and step length by backtracking")
print("Minimizer : ",x)
print("Function values : ",f_x)
print("Number of iteration : ",no_of_iterations)

Newtons method with scaling and step length by backtracking
Minimizer :  [0.         0.03468306]
Function values :  5.788014517642641e-09
Number of iteration :  10


**Observations:** \ 
\
***Newton's method with scaling and constant step length:*** \
Optimizer:  (0,         0.03468306) \
Minimum Value:  5.788014517642641e-09 \
Number of iterations:  10 \

***Newton's Method with Backtracking Line Search With scaling:*** \
Optimizer:  (0,         0.03468306) \
Minimum Value:  5.788014517642641e-09 \
Number of iterations:  10 \

It can be seen that for both the methods (Newton's method with scaling and constant steplength, and Newton's method with scaling and step length by backtracking), the number of iterations required to reach the minimizer is the same (10).

$ \huge{4.}$

In [20]:
x,f_x,no_of_iterations,condition_no = find_minimizer_gd(start,tol,BACKTRACKING_LINE_SEARCH,alpha,rho,gamma)

In [21]:
print("Newtons method without scaling and step length by backtracking")
print("Minimizer : ",x)
print("Function values : ",f_x)
print("Number of iteration : ",no_of_iterations)

Newtons method without scaling and step length by backtracking
Minimizer :  [3.53148711e-10 3.91377499e-02]
Function values :  9.385197325158433e-09
Number of iteration :  5707240


In [22]:
x,f_x,no_of_iterations= find_minimizer_gdscaling(start,tol,BACKTRACKING_LINE_SEARCH_SCALING,alpha,rho,gamma)

In [23]:
print("Newtons method with scaling using a diagonal matrix and step length by backtracking")
print("Minimizer : ",x)
print("Function values : ",f_x)
print("Number of iteration : ",no_of_iterations)

Newtons method with scaling using a diagonal matrix and step length by backtracking
Minimizer :  [0.         0.03967984]
Function values :  9.916069790614023e-09
Number of iteration :  3803


**Observations:** \
\
**Gradient Descent without scaling and step length by backtracking:** \
Optimizer:  (3.53148711e-10, 3.91377499e-02) \
Minimum Value:  9.385197325158433e-09 \
Number of iterations:  5707240 \

**Gradient Descent with scaling using a diagonal matrix and step length by backtracking:** \
Optimizer:  (0,         0.03967984) \
Minimum Value:  9.916069790614023e-09 \
Number of iterations:  3803 \


It can be seen that the number of iterations required to reach the minimizer through gradient descent without scaling and step length by backtracking is 5707240. This is quite large as compared to the iterations needed to reach the minimizer by Gradient Descent with scaling using a diagonal matrix and step length by backtracking, which is 3803. \
Further in the 'with scaling' method, the $x_1$ variable is able to achive its true value of 0, although it is not the case in the 'without scaling' method.
